In [1]:
import sys
sys.path.append('/Users/zhenxi/Desktop/for_git/AIB_PJ1/AIB_PJ1_QnA-chatbot/chatbot')

import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
from utils.Preprocess import Preprocess

불러올 파일은 아래와 같은 형식으로 생겼다.   

; 가락지빵 주문 하고싶어요   
$<가락지빵:FOOD> 주문 하고싶어요   
1	가락지빵	NNG	B_FOOD   
2	주문	NNP	O   
3	하	VV	O   
4	고	EC	O   
5	싶	VX	O   
6	어요	EC	O   

In [4]:
# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$': # $일 경우에 this_sent 리스트 생성
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n': # 다음 줄이 없을 경우, this sent를 sent에 append
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

In [2]:
# 전처리 객체 생성
p = Preprocess(word2index_dic='../../train_tools/dict/chatbot_dict.bin',
               userdic='../../utils/user_dic.tsv')


In [5]:
# 학습용 말뭉치 데이터를 불러옴
corpus = read_file('ner_train.txt')

In [7]:
corpus[:2]

[[('1', '가락지빵', 'NNG', 'B_FOOD'),
  ('2', '주문', 'NNP', 'O'),
  ('3', '하', 'VV', 'O'),
  ('4', '고', 'EC', 'O'),
  ('5', '싶', 'VX', 'O'),
  ('6', '어요', 'EC', 'O')],
 [('1', '가락지빵', 'NNG', 'B_FOOD'),
  ('2', '먹', 'VV', 'O'),
  ('3', '고', 'EC', 'O'),
  ('4', '싶', 'VX', 'O'),
  ('5', '어요', 'EC', 'O')]]

In [9]:
# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])
    sentences.append(sentence)
    tags.append(bio_tag)

In [10]:
print("샘플 크기 : \n", len(sentences))
print("0번째 샘플 단어 시퀀스 : \n", sentences[0])
print("0번째 샘플 bio 태그 : \n", tags[0])
print("샘플 단어 시퀀스 최대 길이 : ", max(len(l) for l in sentences))
print("샘플 단어 시퀀스 평균 길이 : ", (sum(map(len, sentences)) / len(sentences)))

샘플 크기 : 
 61999
0번째 샘플 단어 시퀀스 : 
 ['가락지빵', '주문', '하', '고', '싶', '어요']
0번째 샘플 bio 태그 : 
 ['B_FOOD', 'O', 'O', 'O', 'O', 'O']
샘플 단어 시퀀스 최대 길이 :  168
샘플 단어 시퀀스 평균 길이 :  8.796238649010467


In [13]:
# 토크나이저 정의
# 단어 시퀀스 (sentences)의 경우, Preprocess객체 (인스턴스 명 p)에서 생성하므로 BIO용 토크나이저 객체만 생성한다. 
tag_tokenizer = preprocessing.text.Tokenizer(lower=False)  # 태그 정보는 lower=False 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)

In [18]:
# tokenized 된 단어 인덱스를 dic형태로 확인
tag_tokenizer.word_index

{'O': 1,
 'B_DT': 2,
 'B_FOOD': 3,
 'I': 4,
 'B_OG': 5,
 'B_PS': 6,
 'B_LC': 7,
 'NNP': 8,
 'B_TI': 9}

In [14]:
# 단어 사전 및 태그 사전 크기
vocab_size = len(p.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print("BIO 태그 사전 크기 :", tag_size)
print("단어 사전 크기:", vocab_size)

BIO 태그 사전 크기 : 10
단어 사전 크기: 17751


In [19]:
# 학습용 단어 시퀀스 생성
# x_train : p의 인덱스 시퀀스 이용
# y_train : 방금 만든 tag_tokenizer의 texts_to_sequences이용 
x_train = [p.get_wordidx_sequence(sent) for sent in sentences]
y_train = tag_tokenizer.texts_to_sequences(tags)

In [21]:
x_train[:5]

[[1, 3, 2, 10, 11, 9],
 [1, 233, 10, 11, 9],
 [1, 2, 10, 11, 9],
 [1, 99, 2, 9],
 [1, 28, 22, 2, 50, 21, 18, 1]]

In [22]:
y_train[:5]

[[3, 1, 1, 1, 1, 1],
 [3, 1, 1, 1, 1],
 [3, 1, 1, 1, 1],
 [3, 1, 1, 1],
 [3, 1, 1, 1, 1, 1, 1, 1]]

In [23]:
index_to_ner = tag_tokenizer.index_word  # 시퀀스 인덱스를 NER로 변환하기 위해 사용
index_to_ner[0] = 'PAD' 

In [25]:
index_to_ner # 0: 'PAD'가 추가되었다. 패딩 처리를 위한 밑작업

{1: 'O',
 2: 'B_DT',
 3: 'B_FOOD',
 4: 'I',
 5: 'B_OG',
 6: 'B_PS',
 7: 'B_LC',
 8: 'NNP',
 9: 'B_TI',
 0: 'PAD'}

In [26]:
# 시퀀스 패딩 처리
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

In [28]:
# 학습 데이터와 테스트 데이터를 8:2 비율로 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=.2, random_state=42)


In [29]:
# 출력 데이터를 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

왜 여기서 원-핫 인코딩을 해줬을까?

In [33]:
print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)


학습 샘플 시퀀스 형상 :  (39679, 40)
학습 샘플 레이블 형상 :  (39679, 40, 10)
테스트 샘플 시퀀스 형상 :  (9920, 40)
테스트 샘플 레이블 형상 :  (9920, 40, 10)


In [34]:
# 모델 정의(Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

In [35]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax'))) # tag size는 10 # 10개의 태그중에서 가장 유력한 tag는?
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10)


2021-12-20 23:36:05.424859: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
310/310 [==============================] - 101s 299ms/step - loss: 0.0300 - accuracy: 0.9624
Epoch 2/10
310/310 [==============================] - 122s 395ms/step - loss: 0.0093 - accuracy: 0.9859
Epoch 3/10
310/310 [==============================] - 106s 343ms/step - loss: 0.0064 - accuracy: 0.9901
Epoch 4/10
310/310 [==============================] - 101s 326ms/step - loss: 0.0050 - accuracy: 0.9923
Epoch 5/10
310/310 [==============================] - 102s 330ms/step - loss: 0.0043 - accuracy: 0.9934
Epoch 6/10
310/310 [==============================] - 101s 326ms/step - loss: 0.0035 - accuracy: 0.9946
Epoch 7/10
310/310 [==============================] - 102s 327ms/step - loss: 0.0032 - accuracy: 0.9950
Epoch 8/10
310/310 [==============================] - 102s 328ms/step - loss: 0.0030 - accuracy: 0.9953
Epoch 9/10
310/310 [==============================] - 103s 332ms/step - loss: 0.0027 - accuracy: 0.9956
Epoch 10/10
310/310 [==============================] - 102s 329m

In [36]:
print("평가 결과 : ", model.evaluate(x_test, y_test)[1])
model.save('ner_model.h5')

310/310 [==============================] - 6s 18ms/step - loss: 0.0131 - accuracy: 0.9868
평가 결과 :  0.9868044853210449


index_to_ner의 구성은 아래와 같다.  

{1: 'O',   
 2: 'B_DT',   
 3: 'B_FOOD',   
 4: 'I',   
 5: 'B_OG',   
 6: 'B_PS',   
 7: 'B_LC',   
 8: 'NNP',   
 9: 'B_TI',   
 0: 'PAD'}   

In [37]:
# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):  # 예측값을 index_to_ner를 사용하여 태깅 정보로 변경하는 함수
    result = []
    for sequence in sequences:  # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
        temp = []
        for pred in sequence:  # 시퀀스로부터 예측값을 하나씩 꺼낸다.
            pred_index = np.argmax(pred)  # 예를 들어 [0, 0, 1, 0, 0]이라면 1의 인덱스인 2를 리턴한다.
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))  # 'PAD'는 'O'으로 변경
        result.append(temp)
    return result


In [38]:
# F1 스코어 계산을 위해 사용
from seqeval.metrics import f1_score, classification_report


In [39]:
# 테스트 데이터셋의 NER 예측
y_predicted = model.predict(x_test)
pred_tags = sequences_to_tag(y_predicted)  # 예측된 NER
test_tags = sequences_to_tag(y_test)  # 실제 NER

In [43]:
y_predicted

array([[[7.8983929e-12, 2.7605989e-07, 4.7287180e-10, ...,
         5.8174928e-12, 1.6978555e-10, 1.6601607e-14],
        [2.0193760e-09, 1.0000000e+00, 4.3051166e-08, ...,
         3.1168824e-11, 5.7973626e-10, 2.6091318e-10],
        [8.9779441e-11, 1.0000000e+00, 2.5695335e-10, ...,
         6.2441802e-11, 6.0581304e-11, 9.1424368e-11],
        ...,
        [9.4902068e-02, 1.3850881e-01, 9.3790121e-02, ...,
         9.9203654e-02, 9.5212772e-02, 9.3180500e-02],
        [9.4902068e-02, 1.3850881e-01, 9.3790121e-02, ...,
         9.9203654e-02, 9.5212772e-02, 9.3180500e-02],
        [9.4902068e-02, 1.3850881e-01, 9.3790121e-02, ...,
         9.9203654e-02, 9.5212772e-02, 9.3180500e-02]],

       [[5.3772221e-12, 5.2943898e-07, 2.8654762e-10, ...,
         6.7041984e-12, 1.3434828e-10, 3.8472338e-15],
        [1.4890724e-11, 1.0000000e+00, 2.4179037e-10, ...,
         2.5205816e-12, 7.7784594e-13, 6.9721263e-13],
        [4.7503832e-12, 1.0000000e+00, 1.8632544e-10, ...,
         1.913

In [41]:
pred_tags

[['B_FOOD',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B_FOOD',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B_FOOD',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B_FOOD',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O

In [42]:
test_tags

[['B_FOOD',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B_FOOD',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B_FOOD',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B_FOOD',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O

In [40]:
# F1 평가 결과
print(classification_report(test_tags, pred_tags))
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

/Users/zhenxi/opt/anaconda3/envs/PJ1-2/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_FOOD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/zhenxi/opt/anaconda3/envs/PJ1-2/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/zhenxi/opt/anaconda3/envs/PJ1-2/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/zhenxi/opt/anaconda3/envs/PJ1-2/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/zhenxi/opt/anaconda3/envs/PJ1-2/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NNP seems not to be NE tag.
  wa

              precision    recall  f1-score   support

          NP       1.00      1.00      1.00       281
           _       0.56      0.54      0.55       451
         _DT       1.00      1.00      1.00     10662
       _FOOD       1.00      1.00      1.00      9360
         _LC       0.74      0.59      0.66       239
         _OG       0.62      0.49      0.55       373
         _PS       0.62      0.47      0.53       319
         _TI       0.80      0.80      0.80        40

   micro avg       0.98      0.97      0.97     21725
   macro avg       0.79      0.74      0.76     21725
weighted avg       0.97      0.97      0.97     21725

F1-score: 97.2%
